In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

### 3 techniques of feature selection methods:
1. Filter methods(6)
    - missing values ratio
    - variance threshold
    - correlation coefficient
    - chi-square test of independent (for categorical data)
    - mutual info (for both regression & classification)
    - analysis of variance (ANOVA)
2. Wrapper methods
    - sequential feature selection
    - recursive feature elimination (RFE)
3. Embedded methods
    - L1 (LASSO) regularization
    - Tree model (for regression & classification)

#### With filter methods, 

we primarily apply a statistical measure that suits our data to assign each feature column a calculated score. Based on that score, it'll be decided whether that feature will be kept or removed from our predictive model

Pros and cons:
- Pros: computationally inexpensive + best for eliminating redundant irrelevant features
- Cons: don't take feature correlations into consideration since they work independently on each feature

2 types of filter methods:
1. univariate filter: work on ranking a single feature 
2. multivariate filer: evaluate the entire feature space

#### In wrapper methods,

we primarily choose a subset of features and train them using a ML algorithm. Based on the inferences from this model, we employ a search strategy to look through the space of possible feature subsets and decide which feature to add or remove for the next model development.

This loop continues until the model performance no longer changes with the desired count of features. (k_features)

Pros and cons:
- Pros: takes care of the interactions between features, ultimately finding the optimal subset of features for your model with the lowest possible error.
- Cons: computationally expensive as the features increase

#### In embedded methods,

combining the functionalities of both Filter and Wrapper methods

Embedded methods perform feature selection during the process of training (which is why they're called embedded)

**Upside:** the computational speed is as good as of filter methods + better accuracy